# Inventa: a computational tool to discover structural novelty in natural  extracts libraries


In [1]:
from __future__ import print_function
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import sys 
import lineup_widget
from ipywidgets import *
!jupyter nbextension enable --py --sys-prefix lineup_widget

sys.path.append('../src')
sys.path.append('../gnps_postprocessing/src') 

from import_data import*
from process_data import *
from AC import *
from LC import *
from SC import *
from CC import *
from plot import *

Enabling notebook extension lineup_widget/extension...
      - Validating: OK


# Paths and parameters to define

In [ ]:
# Suffixes necessary for the job: 

repository_path= '/mnt/c/Users/quirosgu/Desktop/Indiv_PF1600/'  # The path were you want your folder to be placed
#quant_table_suffix ='_quant_pos.csv'
spectra_suffix= '_features_ms2_pos.mgf'
#metadata_sample_suffix ='_metadata.tsv'
#isdb_sample_suffix = '_isdb_matched_pos_repond.tsv'
#sirius_sample_suffix = 'compound_identifications_adducts.tsv'
canopus_sample_suffix = 'npc_summary.csv' #'_summary_adducts.tsv' #'canopus_summary.tsv'
#memo_sample_suffix= '_memo_pos.csv'
file_extention = '.mzXML'
ionization_mode = 'pos'

# metadata headers

sampletype_header = 'sample_type'
species_column = 'organism_species'
genus_column = 'organism_genus'
family_column = 'organism_family'
filename_header = 'sample_id'#'ms_filename'
organe_column = 'organism_organe'


#quantitative table
data_process_origin = 'MZMine2' #'MZMine2'
use_ion_identity= False  #False

#Annotation component 

intensity_filter  = True
quantile_filter = True

min_threshold = 0.002
quantile_threshold = 0.75


## cut-offs: 
min_score_final = 0.3             #cut-off filter for considering an isdb annotation valable. You must be extremenly carefull with this parameter, '0.0' as default.
min_ZodiacScore = 0.9             #cut-off filter for considering a sirius annotation valable. It is used in combination with min_ConfidenceScore.
min_ConfidenceScore= 0.25         #cut-off filter for considering a sirius annotation valable. '0.0' as default.

#Literature_component

LC_component = True               # LC will be calculated

max_comp_reported_sp = 10          # max number of compounds reported at species level, more than this value, the plant is considered less interesting
max_comp_reported_g = 50         # max number of compounds reported at genus level,more than this value, the plant is considered less interesting
max_comp_reported_f = 500           # max number of compounds reported at genus level,more than this value, the plant is considered less interesting

#weight for each taxonomic level 
ws = 1
wg = 1
wf = 1 

#Similarity_component

SC_component = True                # SC will be calculated

#Class_component

CC_component =  True              # CC will be calculated
min_class_confidence = 0.8       #cut-off filter for considering a sirius class valable. It is used in combination with min_recurrence.
min_recurrence = 5              # minimum recurrence of a chemical class to consider it acceptable

#specify the weight to modulate each component 
w1 = 1           # 1 means the value itself is taken into account. A 0.5 means onle half of the calculated value is taken into account
w2 = 1
w3 = 1
w4 = 1


### Load Metadata from individual files

In [ ]:
metadata_df = get_metadata_ind_files(repository_path)
metadata_df.head()

100%|██████████| 1921/1921 [00:10<00:00, 183.00it/s]


,sample_id,sample_type,sample_substance_name,organism_kingdom,organism_phylum,organism_class,organism_order,organism_family,organism_genus,organism_species,...,pos_injection_date,bio_leish_donovani_10ugml_inhibition,bio_leish_donovani_2ugml_inhibition,bio_tryp_brucei_rhodesiense_10ugml_inhibition,bio_tryp_brucei_rhodesiense_2ugml_inhibition,bio_tryp_cruzi_10ugml_inhibition,bio_l6_cytotoxicity_10ugml_inhibition,sample_filename_neg,neg_injection_date,massive_id
0,VGF138_A01,qc,qc_mix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2017-10-27,NaN,NaN,NaN,NaN,NaN,NaN,VGF138_A01_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A02,sample,V111819GP-01,Plantae,Tracheophyta,Magnoliopsida,Saxifragales,Paeoniaceae,Paeonia,Paeonia suffruticosa,...,2017-10-27,16.0,10.2,4.8,0.0,8.7,4.2,VGF138_A02_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A03,sample,V111988GP-01,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Scrophulariaceae,Buddleja,Buddleja officinalis,...,2017-10-27,36.7,12.2,9.5,6.0,0.0,0.5,VGF138_A03_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A04,sample,V112020GP-01,Plantae,Tracheophyta,Magnoliopsida,Rosales,Moraceae,Ficus,Ficus tikoua,...,2017-10-27,20.9,10.4,23.6,4.7,1.1,22.5,VGF138_A04_neg.mzXML,2017-10-27,MSV000087728
0,VGF138_A05,sample,V112033GP-01,Plantae,Tracheophyta,Magnoliopsida,Solanales,Solanaceae,Nicandra,Nicandra physalodes,...,2017-10-27,5.7,6.7,14.1,7.7,2.5,7.5,VGF138_A05_neg.mzXML,2017-10-27,MSV000087728


In [ ]:
#if you need to create an unique identifier column like Species|part, use as model the followin line. IF the colum is PRESENT, then don't run it.
metadata_df['organism_sppart'] = metadata_df[species_column]+ "|" + metadata_df[organe_column].map(str)
metadata_df.head(5)

,sample_id,sample_type,sample_substance_name,organism_kingdom,organism_phylum,organism_class,organism_order,organism_family,organism_genus,organism_species,...,bio_leish_donovani_10ugml_inhibition,bio_leish_donovani_2ugml_inhibition,bio_tryp_brucei_rhodesiense_10ugml_inhibition,bio_tryp_brucei_rhodesiense_2ugml_inhibition,bio_tryp_cruzi_10ugml_inhibition,bio_l6_cytotoxicity_10ugml_inhibition,sample_filename_neg,neg_injection_date,massive_id,organism_sppart
0,VGF138_A01,qc,qc_mix,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,VGF138_A01_neg.mzXML,2017-10-27,MSV000087728,NaN
0,VGF138_A02,sample,V111819GP-01,Plantae,Tracheophyta,Magnoliopsida,Saxifragales,Paeoniaceae,Paeonia,Paeonia suffruticosa,...,16.0,10.2,4.8,0.0,8.7,4.2,VGF138_A02_neg.mzXML,2017-10-27,MSV000087728,Paeonia suffruticosa|leaves
0,VGF138_A03,sample,V111988GP-01,Plantae,Tracheophyta,Magnoliopsida,Lamiales,Scrophulariaceae,Buddleja,Buddleja officinalis,...,36.7,12.2,9.5,6.0,0.0,0.5,VGF138_A03_neg.mzXML,2017-10-27,MSV000087728,Buddleja officinalis|leaves
0,VGF138_A04,sample,V112020GP-01,Plantae,Tracheophyta,Magnoliopsida,Rosales,Moraceae,Ficus,Ficus tikoua,...,20.9,10.4,23.6,4.7,1.1,22.5,VGF138_A04_neg.mzXML,2017-10-27,MSV000087728,Ficus tikoua|multiple
0,VGF138_A05,sample,V112033GP-01,Plantae,Tracheophyta,Magnoliopsida,Solanales,Solanaceae,Nicandra,Nicandra physalodes,...,5.7,6.7,14.1,7.7,2.5,7.5,VGF138_A05_neg.mzXML,2017-10-27,MSV000087728,Nicandra physalodes|fruits


In [ ]:
sppart_column = 'organism_sppart'
col_id_unique = filename_header #'organism_sppart'  #column containing an unique identifier for each sample, like Species_plantpart, Species_solvent. It could be the filename

# Start calculation the diferent components

# Annotation Component (FC)

#### AC.1. Process, clean and merge the quant tables, sirius and isdb annotations

In [ ]:
ind_quant_table_full(repository_path, ionization_mode, data_process_origin, file_extention, use_ion_identity, min_score_final, min_ConfidenceScore, min_ZodiacScore)

100%|██████████| 1921/1921 [01:35<00:00, 20.21it/s]

Result are in : /mnt/c/Users/quirosgu/Desktop/Indiv_PF1600/results/VGF159_H11_pos_quant_annotations.tsv


#### AC.2. Calculate the annotation rate of each sample

In [ ]:
AC = annotation_component(repository_path, ionization_mode, file_extention, intensity_filter, quantile_filter, min_threshold, quantile_threshold, filename_header, metadata_df, species_column, genus_column, family_column, sppart_column)
AC.head()

100%|██████████| 1921/1921 [00:16<00:00, 113.32it/s]


,sample_id,organism_species,organism_genus,organism_family,organism_sppart,initial_features,features_after_filtering,Annot_features_after_filtering,AC
0,VGF138_A02,Paeonia suffruticosa,Paeonia,Paeoniaceae,Paeonia suffruticosa|leaves,343,91,48,0.5
1,VGF138_A03,Buddleja officinalis,Buddleja,Scrophulariaceae,Buddleja officinalis|leaves,514,99,51,0.5
2,VGF138_A04,Ficus tikoua,Ficus,Moraceae,Ficus tikoua|multiple,498,75,47,0.4
3,VGF138_A05,Nicandra physalodes,Nicandra,Solanaceae,Nicandra physalodes|fruits,697,64,34,0.5
4,VGF138_A06,Asemeia extraaxillaris,Asemeia,Polygalaceae,Asemeia extraaxillaris|roots,522,87,38,0.6


# Literature Component (LC)


#### LC.1. LC computation

In [ ]:
LC = literature_component(LC_component, repository_path, metadata_df, filename_header, species_column, genus_column, family_column, max_comp_reported_sp, max_comp_reported_g, max_comp_reported_f, ws, wg, wf)
LC.head()

,sample_id,organism_family,organism_genus,organism_species,Reported_comp_Species,Reported_comp_Genus,Reported_comp_Family,LC
0,VGF138_A02,Paeoniaceae,Paeonia,Paeonia suffruticosa,271.0,1058.0,1058.0,0.49624
1,VGF138_A03,Scrophulariaceae,Buddleja,Buddleja officinalis,133.0,557.0,2301.0,0.70958
2,VGF138_A04,Moraceae,Ficus,Ficus tikoua,0.0,895.0,6850.0,0.68400
3,VGF138_A05,Solanaceae,Nicandra,Nicandra physalodes,6.0,6.0,13616.0,0.72048
4,VGF138_A06,Polygalaceae,Asemeia,Asemeia extraaxillaris,0.0,74.0,1434.0,0.95652


# Similarity component (SC)

#### SC.1. Calculate MEMO matrix from individual files

In [ ]:
#calculate MEMO matrix from individual files
metric_df = calculate_memo_matrix_ind_files(repository_path, ionization_mode, spectra_suffix, filename_header)
#metric_df.head()

100%|██████████| 1920/1920 [18:42<00:00,  1.71it/s]


Computing MEMO matrix from unaligned samples took: 1167.078125 seconds


/home/quirosgu/anaconda3/envs/inventa/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
def drop_samples_based_on_string_ind(repository_path, ionization_mode, filename_header, sampletype_header,list_of_strings_for_QC_Blank_filter, column):
    """ drop samples based on string 
    Args:
        pd dataframe
        list of string

    Returns:
        pd dataframe
    """
    path = os.path.normpath(repository_path)
    samples_dir = [directory for directory in os.listdir(path)]

    for directory in tqdm(samples_dir):
   
        MEMO_path = os.path.join(path +'/results/', 'memo_matrix_non_filtered' +'_'+ ionization_mode + '.tsv')
        metadata_path = os.path.join(path +'/results/', 'Metadata_combined.tsv')

        try:
            metric_df = pd.read_csv(MEMO_path, sep='\t')
            metadata_df = pd.read_csv(metadata_path, sep='\t')
            
        except FileNotFoundError:
            continue
        except NotADirectoryError:
            continue

    if os.path.isfile(MEMO_path):
        metric_df = pd.read_csv(MEMO_path, sep='\t')
        metadata_df = pd.read_csv(metadata_path, sep='\t')

        df= pd.merge(metric_df, metadata_df[[filename_header, sampletype_header]], how='left', on=filename_header)
        print(df.shape)

        for string in list_of_strings_for_QC_Blank_filter:
            df = df[~df[column].str.contains(string, na=False)]
            df = df.dropna(how = 'any', subset=[column])
        print(df.shape)
    
    df.drop(sampletype_header, axis=1, inplace=True)
    
    pathout = os.path.join(path, 'results/')
    os.makedirs(pathout, exist_ok=True)
    pathout = os.path.join(pathout, 'memo_matrix_filtered' +'_' + ionization_mode + '.tsv')
    df.to_csv(pathout, sep ='\t')

    return df

In [ ]:
#remove experimental controls and blancks
list_of_strings_for_QC_Blank_filter = ['blank', 'qc'] #erase all the blanks and QC's - Change the string as needed
column_to_use_for_filtering = sampletype_header #this information should be included in the metadata table
metric_df= drop_samples_based_on_string_ind(repository_path, ionization_mode, filename_header, sampletype_header,list_of_strings_for_QC_Blank_filter, column_to_use_for_filtering)
#metric_df.head()

  0%|          | 4/1921 [21:10<169:05:24, 317.54s/it]


KeyboardInterrupt: 

#### SC.2. SC calculation

In [ ]:
def similarity_component_ind(repository_path, ionization_mode, filename_header):
    """ function to compute the similarity component based on the MEMO matrix and machine learning unsupervised clustering methods 
    Args:
        df = memo matrix

    Returns:
        None
    """
    path = os.path.normpath(repository_path)
    samples_dir = [directory for directory in os.listdir(path)]

    for directory in tqdm(samples_dir):
   
        MEMO_path = os.path.join(path +'/results/', 'memo_matrix_filtered' +'_'+ ionization_mode + '.tsv')

        try:
            df = pd.read_csv(MEMO_path, sep='\t')

        except FileNotFoundError:
            continue
        except NotADirectoryError:
            continue

    if os.path.isfile(MEMO_path):

        df = pd.read_csv(MEMO_path, sep='\t')
        df1 = df.copy()
        df1.set_index(filename_header, inplace=True)

        df2 = df.copy()
        
        #specify the parameters of the individual classification algorithms
        clf = IsolationForest(n_estimators=100, 
                    max_samples='auto', 
                    contamination= 'auto', #0.15,
                    max_features=1.0, 
                    bootstrap=False, 
                    n_jobs=None, 
                    random_state=None)
        clf.fit(df1)
        pred = clf.predict(df1)
        df1['anomaly_IF'] = pred
        outliers = df1.loc[df1['anomaly_IF']==-1]
        outlier_index = list(outliers.index)

        lof = LocalOutlierFactor(n_neighbors=10, 
                            algorithm='auto',
                            leaf_size=30,
                            metric='braycurtis', 
                            contamination= 0.15,
                            novelty=False, 
                            n_jobs=None)#-1)
        df1['anomaly_LOF'] = lof.fit_predict(df1)
        outliers = df1.loc[df1['anomaly_LOF']==-1]
        outlier_index = list(outliers.index)

        ocsvm = OneClassSVM(kernel='rbf', 
                            degree=3, 
                            gamma='scale', 
                            tol= 1e-3, 
                            max_iter=-1, 
                            nu=0.01)
        df1['anomaly_OCSVM'] = ocsvm.fit_predict(df1)
        outliers = df1.loc[df1['anomaly_OCSVM']==-1]
        outlier_index = list(outliers.index)

        #recover and print the results
        df1.reset_index(inplace=True)

        df = pd.merge(df1,df2, how='left', on =filename_header)

        df = df[[filename_header, 'anomaly_IF', 'anomaly_LOF', 'anomaly_OCSVM']]

        def similarity_conditions(df):
            if (df['anomaly_IF'] == -1) | (df['anomaly_LOF'] == -1) | (df['anomaly_OCSVM'] == -1):
                return 1
            else: 
                return 0 

        df['SC'] = df.apply(similarity_conditions, axis=1)
       
        path = os.path.normpath(repository_path)
        pathout = os.path.join(path, 'results/')
        os.makedirs(pathout, exist_ok=True)
        pathout = os.path.join(pathout, 'Similarity_component_results' +'_' + ionization_mode + '.tsv')
        df.to_csv(pathout, sep ='\t')
        return df
    else:
        print('Similarity component not calculated')

In [ ]:
SC = similarity_component_ind(repository_path, ionization_mode, metric_df, SC_component, filename_header)
SC.head(10)

  0%|          | 0/1921 [00:29<?, ?it/s]


KeyboardInterrupt: 

# Class component (CC)

In [ ]:
def class_component_ind_files_PF1600(CC_component, repository_path, ionization_mode, min_class_confidence, metadata_df, filename_header, species_column, genus_column, family_column):
    """
    Function to recover the chemical classes predicted and reported from individual files and LOTUS accordingly, used for calculation of inventa non aligned data
    """
    if CC_component == True:
        
        path = os.path.normpath(repository_path)
        samples_dir = [directory for directory in os.listdir(path)]

        df =pd.DataFrame()
        files = []
        for directory in tqdm(samples_dir):
            canopus_path = os.path.join(path, directory, ionization_mode, directory + '_WORKSPACE_SIRIUS', 'npc_summary.csv')
            #canopus_path = os.path.join(path, directory, ionization_mode, directory + '_WORKSPACE_SIRIUS', 'canopus_summary_adducts.tsv')
            try:
                canopus_df =pd.read_csv(canopus_path, sep=',')
            except FileNotFoundError:
                continue
            except NotADirectoryError:
                continue

            #recover filenames 
            files.append(directory)
            
            #read canpus filename
            canopus_df =pd.read_csv(canopus_path, sep=',')
            canopus_df['partial_filename'] = canopus_df['directoryName'].apply(lambda r: '/'.join(r.split('/')[8:]) if len(r.split('_')) > 1 else r)
            canopus_df['partial_filename'] = canopus_df['partial_filename'].apply(lambda r: '_'.join(r.split('_')[:-3]) if len(r.split('_')) > 1 else r)
            canopus_df['partial_filename'] = canopus_df['partial_filename'].apply(lambda r: '_'.join(r.split('_')[1:]) if len(r.split('_')) > 1 else r)
            canopus_df.rename(columns={'name': 'shared name','classProbability': 'NPC_class_Probability', 'class':'NPC#class'}, inplace=True)

            canopus_df = canopus_df[['partial_filename', 'shared name', 'NPC#class', 'NPC_class_Probability']]
            canopus_df = canopus_df[canopus_df.NPC_class_Probability > min_class_confidence]
            canopus_df = canopus_df[['partial_filename', 'NPC#class']].groupby('partial_filename').agg(set)
            df = df.append(canopus_df, ignore_index=False)

        df.reset_index(inplace=True)
        df = pd.merge(df, metadata_df[[filename_header, species_column]], how= 'left', left_on='partial_filename', right_on=filename_header)
        df.drop('partial_filename', axis=1, inplace=True)

        ## Retrieve classes reported in Lotus
        LotusDB = pd.read_csv('../data_loc/LotusDB_inhouse_metadata.csv',sep=',').dropna()
        
        #create a set of species present in the metatada and reduce the lotus DB to it
        set_sp = set(metadata_df[species_column].dropna())
        LotusDB= LotusDB[LotusDB['organism_taxonomy_09species'].isin(set_sp)]

        #retrieve the chemical classes associated to the species and genus
        ccs = LotusDB[['organism_taxonomy_09species', 'structure_taxonomy_npclassifier_03class']].groupby('organism_taxonomy_09species').agg(set)
        ccs.rename(columns={'structure_taxonomy_npclassifier_03class': 'Chemical_class_reported_in_species'}, inplace=True)
        ccg = LotusDB[['organism_taxonomy_08genus', 'structure_taxonomy_npclassifier_03class']].groupby('organism_taxonomy_08genus').agg(set)
        ccg.rename(columns={'structure_taxonomy_npclassifier_03class': 'Chemical_class_reported_in_genus'}, inplace=True)

        #merge into a single dataframe
        cc = pd.merge(metadata_df[[filename_header, species_column, genus_column, family_column]], ccs, left_on= species_column, right_on='organism_taxonomy_09species', how='left')
        cc = pd.merge(cc,ccg, left_on= genus_column, right_on='organism_taxonomy_08genus', how='left')

        #obtain the difference between the predicted and reported compounds
        #first merge both dataframes
        df = pd.merge(df[[filename_header, 'NPC#class']],cc,on=filename_header, how='left')#.dropna()

        #check if the chemical classes from Sirius are reported in the species
        df['New_CC_in_sp'] = df["NPC#class"] - df["Chemical_class_reported_in_species"]  
        df['New_CC_in_genus'] = df["New_CC_in_sp"] - df["Chemical_class_reported_in_genus"]

        #function to check if there are new cc in ccs/ccg
        def is_empty(df):
                    """ function to check if the sets are empty or not 
                        Args:
                            df = Class component column CC 
                            Returns:
                            None
                    """
                    if df:
                        return 0.5 # if the column is not empty then 1, something is new in the sp &/ genus
                    else:
                        return 0

        df['CCs'] = df['New_CC_in_sp'].notnull().apply(is_empty)
        df['CCg'] = df['New_CC_in_genus'].notnull().apply(is_empty)
        df['CC'] = df['CCs'] + df['CCg']
        df['CC'] = df['CC'].fillna(1)

        #change all the NaN with a string to indicate lack of reports in the literature 
        df['Chemical_class_reported_in_species'] = df['Chemical_class_reported_in_species'].fillna('nothing in DB')
        df['Chemical_class_reported_in_genus'] = df['Chemical_class_reported_in_genus'].fillna('nothing in DB')
        df['New_CC_in_sp'] = df['New_CC_in_sp'].fillna('nothing in DB')
        df['New_CC_in_genus'] = df['New_CC_in_genus'].fillna('nothing in DB')
        
        #if we considered lack of reports in DB... all the proposed cc will be new to that particular species, hence.. the real value of CC is 1, not zero
        string = 'nothing in DB'
        df.loc[df['Chemical_class_reported_in_species'] ==string, 'CC'] = 1
       
        pathout = os.path.join(path, 'results/')
        os.makedirs(pathout, exist_ok=True)
        pathout = os.path.join(pathout, 'Class_component_results' +'_' + ionization_mode + '.tsv')
        df.to_csv(pathout, sep ='\t')
        return df
    else:
        print ('No search was done because the Class component is not going to be calculated')

In [ ]:
CC= class_component_ind_files_PF1600(CC_component, repository_path, ionization_mode, min_class_confidence, metadata_df, filename_header, species_column, genus_column, family_column)
CC.head()

100%|██████████| 1921/1921 [00:30<00:00, 62.51it/s]


,sample_id,NPC#class,organism_species,organism_genus,organism_family,Chemical_class_reported_in_species,Chemical_class_reported_in_genus,New_CC_in_sp,New_CC_in_genus,CCs,CCg,CC
0,VGF138_A02,"{Triacylglycerols, Prenyl quinone meroterpenoi...",Paeonia suffruticosa,Paeonia,Paeoniaceae,"{Oligomeric stibenes, Pyrrole alkaloids, Antho...","{Branched fatty acids|Unsaturated fatty acids,...","{Triacylglycerols, Tetraketide meroterpenoids,...","{Triacylglycerols, Tetraketide meroterpenoids,...",0.5,0.5,1.0
1,VGF138_A03,"{Triacylglycerols, Lanostane, Tirucallane and ...",Buddleja officinalis,Buddleja,Scrophulariaceae,"{Acyclic monoterpenoids, Flavones, Cycloartane...","{Humulane sesquiterpenoids, Lanostane, Tirucal...","{Triacylglycerols, Prenyl quinone meroterpenoi...","{Triacylglycerols, Prenyl quinone meroterpenoi...",0.5,0.5,1.0
2,VGF138_A04,"{Triacylglycerols, Isoflavones, Glycosyldiacyl...",Ficus tikoua,Ficus,Moraceae,nothing in DB,"{Isoflavones, Flavones, Lupane triterpenoids, ...",nothing in DB,nothing in DB,0.0,0.0,1.0
3,VGF138_A05,"{Triacylglycerols, Prenyl quinone meroterpenoi...",Nicandra physalodes,Nicandra,Solanaceae,"{Tropane alkaloids, Cycloartane triterpenoids|...","{Tropane alkaloids, Cycloartane triterpenoids|...","{Triacylglycerols, Cholestane steroids, Vitami...","{Triacylglycerols, Cholestane steroids, Vitami...",0.5,0.5,1.0
4,VGF138_A06,"{Triacylglycerols, Isoflavones, Other Octadeca...",Asemeia extraaxillaris,Asemeia,Polygalaceae,nothing in DB,nothing in DB,nothing in DB,nothing in DB,0.0,0.0,1.0


# Priority rank Results

In [ ]:
def priority_score_ind(repository_path, filename_header, ionization_mode, species_column, genus_column, family_column, sppart_column, w1, w2, w3, w4):


    path = os.path.normpath(repository_path)
    samples_dir = [directory for directory in os.listdir(path)]

    for directory in tqdm(samples_dir):
        AC_path = os.path.join(path +'/results/', 'Annotation_component_results' +'_'+ ionization_mode + '.tsv')
        LC_path = os.path.join(path +'/results/', 'Annotation_component_results' +'_'+ ionization_mode + '.tsv')
        SC_path = os.path.join(path +'/results/', 'Annotation_component_results' +'_'+ ionization_mode + '.tsv')
        CC_path = os.path.join(path +'/results/', 'Annotation_component_results' +'_'+ ionization_mode + '.tsv')
        metadata_path = os.path.join(path +'/results/', 'Metadata_combined.tsv')

        try:
            AC = pd.read_csv(AC_path, sep=',')
            LC = pd.read_csv(LC_path, sep='\t')
            SC = pd.read_csv(SC_path, sep='\t')
            CC = pd.read_csv(CC_path, sep='\t')
            metadata_df= pd.read_csv(metadata_path, sep='\t')
            
        except FileNotFoundError:
            continue
        except NotADirectoryError:
            continue


        if os.path.isfile(AC_path):
        
                df = pd.read_csv(AC_path, sep='\t')
                df['PS'] = w1*df['AC']

        else:
                df = pd.read_csv(metadata_path, usecols= [filename_header, family_column, genus_column, species_column, sppart_column], sep='\t')
                
        if os.path.isfile(LC_path):
                
                LC = pd.read_csv(LC_path, sep='\t')
                df =pd.merge(
                    left=df,
                    right=LC[[filename_header, 'LC', 'Reported_comp_Species', 'Reported_comp_Genus', 'Reported_comp_Family']], 
                    how='left', 
                    on=filename_header)
                df['PS'] = w1*df['AC'] + w2*df['LC']
        else:
            df

        if os.path.isfile(SC_path):
            SC = pd.read_csv(SC_path, sep='\t')
            df =pd.merge(
                        left=df,
                        right=SC[[filename_header, 'SC']], 
                        how='left', 
                        on=filename_header)
            df['PS'] = w1*df['AC'] + w2*df['LC'] + w3*df['SC']
        else:
            df

        if os.path.isfile(CC_path):
            CC = pd.read_csv(CC_path, sep='\t')
            df =pd.merge(
                            left=df,
                            right=CC[[filename_header,'CCs','CCg', 'CC', 'New_CC_in_sp', 'New_CC_in_genus']], 
                            how='left', 
                            on =filename_header)
            df['PS'] = w1*df['AC'] + w2*df['LC'] + w3*df['SC'] + w4*df['CC']
        else:
            df
            
    path = os.path.normpath(repository_path)
    pathout = os.path.join(path, 'results/')
    os.makedirs(pathout, exist_ok=True)
    pathout = os.path.join(pathout, 'Priority_score_results' +'_'+ ionization_mode + '.tsv')
    df.to_csv(pathout, sep ='\t')
    return df

In [ ]:
PS = priority_score_ind(repository_path, filename_header, ionization_mode, species_column, genus_column, family_column, sppart_column, w1, w2, w3, w4)
PS.head()

100%|██████████| 1921/1921 [00:28<00:00, 67.07it/s]


UnboundLocalError: local variable 'df' referenced before assignment

### Display results

In [ ]:
#Show the results in an interactive way
def selection_changed(selection):
    return PS.iloc[selection]
interact(selection_changed, selection=lineup_widget.LineUpWidget(PS));

interactive(children=(LineUpWidget(value=[], description='selection', layout=Layout(align_self='stretch', heig…